In [ ]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm

import os
import shutil
import json

import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import albumentations as A
from efficientnet_pytorch import EfficientNet
from pprint import pprint
import timm
import random

import os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

In [ ]:
data_path = '/data/competition/Dacon/Ego_Vision/dataset'

train_path = data_path + '/remove_new_images/train'

hand_gesture = pd.read_csv(data_path + '/hand_gesture_pose.csv')

In [ ]:
json_folders = sorted(glob(data_path + '/train/*'), key = lambda x : int(x.split('/')[-1]))
json_folders[:5]

In [ ]:
train_folders = sorted(glob(train_path + '/*'), key = lambda x : int(x.split('/')[-1]))
train_folders[:5]

In [ ]:
train_folder = train_folders[0]

image_paths = sorted(glob(train_folder + '/*.png'), key = lambda x : int(x.split('/')[-1].replace('.png','')))

In [ ]:
answers = []
for json_folder, train_folder in zip(json_folders, train_folders) :
    json_path = glob(json_folder + '/*.json')[0]
    js = json.load(open(json_path))
    cat = js.get('action')[0]
    cat_name = js.get('action')[1]
    answers.append([train_folder.replace(data_path,''),cat, cat_name])

answers = pd.DataFrame(answers, columns = ['train_path','answer', 'answer_name'])
answers

In [ ]:
num = -1
for i in range(196):
    if i in answers["answer"].values:
        num = num + 1
        for j in range(answers.shape[0]):
            if answers["answer"][j] == i:
                answers["answer"][j] = num

In [ ]:
folder_labels = np.zeros(len(train_folders))
for i in tqdm(range(answers.shape[0])):
    for j in range(len(train_folders)):
        if answers["train_path"][i] in train_folders[j]:
            folder_labels[j] = answers["answer"][i]

In [ ]:
def train_step(batch_item, epoch, batch, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        return loss
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
            
        return loss

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, imgs, labels=None, transformer=None, mode="train"):
        self.imgs = imgs
        self.transformer = transformer
        self.mode = mode
        if self.mode == "train" :
            self.labels = labels
        elif self.mode == "val" :
            self.labels = labels
            
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, i):
        img = cv2.imread(self.imgs[i]).astype(np.float32)/255
        img = cv2.resize(img, dsize=(384,384))
        if self.mode == "train":
            if self.transformer != None:
                img = self.transformer(image=img)
                img = np.transpose(img["image"], (2,0,1))
            else:
                img = np.transpose(img, (2,0,1))
            return {
                "img" : torch.tensor(img, dtype=torch.float32),
                "label" : torch.tensor(self.labels[i], dtype=torch.long)
            }
        elif self.mode == "val":
            img = np.transpose(img, (2,0,1))
            return {
                "img" : torch.tensor(img, dtype=torch.float32),
                "label" : torch.tensor(self.labels[i], dtype=torch.long)
            }
        
        else:
            img = np.transpose(img, (2,0,1))
            return {
                "img" : torch.tensor(img, dtype=torch.float32)
            }

In [ ]:
FOLDS = 6
skf = StratifiedKFold(n_splits=FOLDS, random_state=42, shuffle=True)

device = torch.device("cuda:0")
dropout_rate = 0.1
class_num = 157
learning_rate = 1e-4
BATCH_SIZE = 16
EPOCHS = 15
MODELS = 'efficientnet-b7'
save_path = f"./models/Py_Test_{EPOCHS}"

folder_train_idx = 0
folder_val_idx = 0

n=0

for train_idx, val_idx in skf.split(train_folders, folder_labels):
    n = n + 1
    folder_train_idx = train_idx
    folder_val_idx = val_idx

    train_imgs = []

    for num,i in enumerate(train_folders):
        if num in folder_train_idx:
            train_img = sorted(glob(i+"/*.png"))
            for j in train_img:
                train_imgs.append(j)

    val_imgs = []

    for num,i in enumerate(train_folders):
        if num in folder_val_idx:
            train_img = sorted(glob(i+"/*.png"))
            for j in train_img:
                val_imgs.append(j)
                
    train_labels = np.zeros(len(train_imgs))
    for i in tqdm(range(answers.shape[0])):
        for j in range(len(train_imgs)):
            if answers["train_path"][i] in train_imgs[j]:
                train_labels[j] = answers["answer"][i]

    val_labels = np.zeros(len(val_imgs))
    for i in tqdm(range(answers.shape[0])):
        for j in range(len(val_imgs)):
            if answers["train_path"][i] in val_imgs[j]:
                val_labels[j] = answers["answer"][i]
                
    train_imgs = np.array(train_imgs)
    val_imgs = np.array(val_imgs)

    train_labels = np.array(train_labels)
    val_labels = np.array(val_labels)
    
    albumentations_transform = A.Compose([
        A.GaussianBlur(),
        A.ShiftScaleRotate(),
        A.GaussNoise(),
        A.RandomGamma(),
    ])
    
    model = EfficientNet.from_pretrained(MODELS, advprop=True, num_classes=class_num)
    model._dropout.p = dropout_rate
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, patience=10)
    criterion = nn.CrossEntropyLoss()

    train_dataset = CustomDataset(train_imgs, train_labels, transformer=albumentations_transform)
    val_dataset = CustomDataset(val_imgs, val_labels, transformer=albumentations_transform, mode="val")

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=16, shuffle=True)

    sample_batch = next(iter(train_dataloader))

    loss_plot, val_loss_plot = [], []

    for epoch in range(EPOCHS):
        total_loss, total_val_loss = 0, 0

        tqdm_dataset = tqdm(enumerate(train_dataloader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_loss = train_step(batch_item, epoch, batch, training)
            total_loss += batch_loss

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Total Loss' : '{:06f}'.format(total_loss/(batch+1))
            })
        loss_plot.append(total_loss/(batch+1))

        tqdm_dataset = tqdm(enumerate(val_dataloader))
        training = False
        for batch, batch_item in tqdm_dataset:
            batch_loss = train_step(batch_item, epoch, batch, training)
            total_val_loss += batch_loss

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1))
            })
        val_loss_plot.append(total_val_loss/(batch+1))
        scheduler.step(total_val_loss/(batch+1))

        if np.min(val_loss_plot) == val_loss_plot[-1]:
            torch.save(model, save_path + f"_{n}.pt")
            print("## Model Save")